# Demo's for AZ-2005 (work in progress)

This demo will showcase RAG patterns using Azure Cognitive Search

In [ ]:
#r "nuget:Microsoft.Extensions.Logging, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Debug, 8.0.0"

#r "nuget:Microsoft.SemanticKernel, 1.16.2"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core, 1.16.2-alpha"
#r "nuget:dotenv.net, 3.0.0"

#r "nuget:Microsoft.KernelMemory.Core, 0.69.240727.1"

In [ ]:
using Microsoft.KernelMemory;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.Extensions.Configuration;

var azureOpenAITextConfig = new AzureOpenAIConfig();
var azureOpenAIEmbeddingConfig = new AzureOpenAIConfig();
var azureAISearchConfigWithHybridSearch = new AzureAISearchConfig();

azureAISearchConfigWithHybridSearch.UseHybridSearch = true;

new ConfigurationBuilder()
            .AddJsonFile($"{System.IO.Directory.GetCurrentDirectory()}\\appsettings.dev.json")
            .Build()
            .BindSection("KernelMemory:Services:AzureOpenAIText", azureOpenAITextConfig)
            .BindSection("KernelMemory:Services:AzureOpenAIEmbedding", azureOpenAIEmbeddingConfig)
            .BindSection("KernelMemory:Services:AzureAISearch", azureAISearchConfigWithHybridSearch);

var memory = new KernelMemoryBuilder()
            .WithAzureOpenAITextGeneration(azureOpenAITextConfig)
            .WithAzureOpenAITextEmbeddingGeneration(azureOpenAIEmbeddingConfig)
            .WithAzureAISearchMemoryDb(azureAISearchConfigWithHybridSearch)
            .WithSearchClientConfig(new SearchClientConfig { MaxMatchesCount = 2, Temperature = 0, TopP = 0 })
            .Build<MemoryServerless>();

In [ ]:
// delete index
await memory.DeleteIndexAsync("demo_04");

// Import a file
await memory.ImportDocumentAsync(@"C:\_git\weslbo\AZ-2005\semantic-kernel.pdf", index: "demo_04");

In [ ]:
var answer = await memory.AskAsync("What is Semantic Kernel?");

answer

// foreach (var x in answer.RelevantSources)
// {
//     Console.WriteLine($"  * {x.SourceName} -- {x.Partitions.First().LastUpdate:D}");
// }

In [ ]:
ChatHistory history = [];   // add chatHistory

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

while (true)
{
    Console.Write("User > ");

    var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your message (type EXIT to quit):");

    if (question.ToLower() == "exit" || String.IsNullOrEmpty(question))
    {
        break;
    }

    Console.Write(question);
    Console.WriteLine();

    //var result = chatCompletionService.GetStreamingChatMessageContentsAsync(history, kernel: kernel);  // replace request with history
    var answer = await memory.AskAsync(question, index: "demo_04", minRelevance: 0.76);

    history.AddUserMessage(question);       // add user message to history  
    history.AddAssistantMessage(answer.Result);    // add assistant message to history

    string fullMessage = "";
    Console.Write("Assistant > ");

    await foreach (var content in result)
    {
        Console.Write(content.Content);
        fullMessage += content.Content;
    }

    history.AddAssistantMessage(fullMessage);  // add assistant message to history

    Console.WriteLine();
    Console.WriteLine();
}

In [ ]:
var embeddingQuestion = $"
 ";